In [345]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def loadDataSet2():
    return [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]]

In [346]:
"""
创建集合大小为1的项集
"""
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict  

In [347]:
"""
计算指定项集的支持度
D是数据集合，Ck是要求支持度的项集，minSupport是最小支持度
"""
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.get(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [348]:
"""
生成集合大小比原来的大一的项集
"""
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    # 如果Lk只有一项，则retList为[]
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            # 为了每次添加的都是单个项构成的集合，所以Lk中项的大小是k-1，那么前k-2项相同，才能使合并后只比原来多一项。
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [349]:
"""
求出数据集合中，支持度大于最小支持度的所有项集
"""
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    # 转化为list类型重要
    C1 = list(C1)
    D = map(set, dataSet)
    # 转化为list类型重要
    D = list(D)
#     print(D)
#     print(C1)
    L1, supportData = scanD(D, C1, minSupport)
#     print(L1)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [350]:
"""
计算 freqSet-conseq -> conseq 的可信度， conseq是H中的元素
"""
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [351]:
"""
# 1个元素的集合 -> (len(freqSet) - 1) 元素的集合
# 2个元素的集合 -> (len(freqSet) - 2) 元素的集合
# ...
"""
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    print("H:", H)
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
#         print("Hmp1:",Hmp1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
#         print("Hmp1:",Hmp1)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [352]:
"""
两个元素的集合求关联规则
三个元素的集合求关联规则
...
"""
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        print(L[i])
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            print(H1)
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [353]:
"""
进行项集为1的测试，如果可行再应用到递推中，start
"""

'\n进行项集为1的测试，如果可行再应用到递推中，start\n'

In [354]:
# 加载数据
dataSet=loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [355]:
# 获取集合大小为1的所有项集
C1=createC1(dataSet)
# print(len(C1))
# TypeError: object of type 'map' has no len()
C1 = list(C1)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [356]:
# 将数据类型转换为集合
D=map(set, dataSet)
# print(len(D))
D = list(D)
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [357]:
# 求集合大小为1的项集的支持度，返回留下支持度大于最小支持度的项集和它们的支持度
L1, suppData0 = scanD(D, C1, 0.5)
print(L1)
print(suppData0)

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]
{frozenset({4}): 0.25, frozenset({5}): 0.75, frozenset({2}): 0.75, frozenset({3}): 0.75, frozenset({1}): 0.5}


In [358]:
"""
进行项集为1的测试，如果可行再应用到递推中，end
"""

'\n进行项集为1的测试，如果可行再应用到递推中，end\n'

In [359]:
"""
测试priori算法，start
"""

'\n测试priori算法，start\n'

In [360]:
L1, suppData0 = apriori(dataSet)
print(L1)
print()
print(suppData0)

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})], [frozenset({2, 3, 5})], []]

{frozenset({5}): 0.75, frozenset({3}): 0.75, frozenset({2, 3, 5}): 0.5, frozenset({1, 2}): 0.25, frozenset({1, 5}): 0.25, frozenset({3, 5}): 0.5, frozenset({4}): 0.25, frozenset({2, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({1}): 0.5, frozenset({1, 3}): 0.5, frozenset({2}): 0.75}


In [361]:
L1, suppData0 = apriori(dataSet, minSupport=0.7)
print(L1)
print()
print(suppData0)

[[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})], []]

{frozenset({5}): 0.75, frozenset({3}): 0.75, frozenset({3, 5}): 0.5, frozenset({4}): 0.25, frozenset({2, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({1}): 0.5, frozenset({2}): 0.75}


In [362]:
"""
测试priori算法，end
"""

'\n测试priori算法，end\n'

In [363]:
"""
测试关联规则start
"""

'\n测试关联规则start\n'

In [364]:
L, suppData = apriori(dataSet, minSupport=0.5)
print(L)
print()
print(suppData)

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})], [frozenset({2, 3, 5})], []]

{frozenset({5}): 0.75, frozenset({3}): 0.75, frozenset({2, 3, 5}): 0.5, frozenset({1, 2}): 0.25, frozenset({1, 5}): 0.25, frozenset({3, 5}): 0.5, frozenset({4}): 0.25, frozenset({2, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({1}): 0.5, frozenset({1, 3}): 0.5, frozenset({2}): 0.75}


In [365]:
rules = generateRules(L, suppData, minConf=0.7)

[frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})]
[frozenset({3}), frozenset({5})]
[frozenset({1}), frozenset({3})]
frozenset({1}) --> frozenset({3}) conf: 1.0
[frozenset({2}), frozenset({5})]
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
[frozenset({2}), frozenset({3})]
[frozenset({2, 3, 5})]
[frozenset({2}), frozenset({3}), frozenset({5})]
H: [frozenset({2}), frozenset({3}), frozenset({5})]
Hmp1: [frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]
Hmp1: []
[]


In [366]:
rules = generateRules(L, suppData, minConf=0.5)
rules

[frozenset({3, 5}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3})]
[frozenset({3}), frozenset({5})]
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
[frozenset({1}), frozenset({3})]
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
[frozenset({2}), frozenset({5})]
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
[frozenset({2}), frozenset({3})]
frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
[frozenset({2, 3, 5})]
[frozenset({2}), frozenset({3}), frozenset({5})]
H: [frozenset({2}), frozenset({3}), frozenset({5})]
Hmp1: [frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666

[(frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [367]:
"""
测试关联规则end
"""

'\n测试关联规则end\n'

In [368]:
help(dict)

Help on class dict in module builtins:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if D has a key k, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |